In this notebook, I am going to demonstrate the use of HuggingFace Sentence Transformers to extract a phrase from a minute book and provide a list of pages that contain the phrase ( using a threshold of .5 )

We use cosine similarity to derive insights.

The minute book has already been converted to text using PyPDF2 and is saves as extracted.txt .

In figuring out the pretrained model that would work best for our use case , we tried several models from huggingface.

The BERT model bert-base-nli-mean-tokens under Sentence Transformers is deprecated, so we tried others.

The model can be served with the Accelerated Inference API. 

Sentence Transformers may not be the solution for our use case. 

We will try Word Embeddings too.

In this notebook, we use Tensorflow but it is possible to use pytorch as well. 






In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#import libraries
%tensorflow_version 1.x
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import os, sys
import time
from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint

TensorFlow 1.x selected.


In [4]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [5]:
!pip install -U sentence-transformers

     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 2.6 MB 13.7 MB/s 
     |████████████████████████████████| 1.2 MB 45.9 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 895 kB 60.0 MB/s 
     |████████████████████████████████| 636 kB 53.3 MB/s 
     |████████████████████████████████| 3.3 MB 31.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=2047ccac5c58d821514cf767919cd11f4f00de40808135c223855a631f3c2cfd
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
from sentence_transformers import SentenceTransformer
model_name = "sentence-transformers/paraphrase-distilroberta-base-v2"
model_para_distilroberta = SentenceTransformer('sentence-transformers/paraphrase-distilroberta-base-v2')

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/686 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Import  data 

In [7]:
#sample data 
import os

base_path = '/content/drive/MyDrive/week5/files'
save_path = '/content/drive/MyDrive/week5/files'
os.chdir(base_path)
os.getcwd()

os.listdir()
file_text = open(os.path.join(base_path, "extracted.txt"),"r")
file_list = file_text.readlines()
file_list
#list of strings 

["O2COPL)\\pRoTSSORYOTEDue: On Demand Dated: JulY Ist, L975$63,364.25 R. R. #3Woodstock, OntarioON DEMAND, I promise to pay CHRISTOPHER ECONOMOPOULOS----the sum of SIXtY-THREE THOUSAND, THREE HUNDRED AND SIXTY-FOUR---- ($o:,364.25)--- 25/100 DoLLARStogether with interest at the rate of nil per cent perannum. Value receive'd.GRECO-IBERIAN CERAMICS LIMITED(r*,U3Per:ct,ll\n",
 '!=om(nG)o@\n',
 'oo-to\n',
 '\n',
 'For Minartry Ur. OntyA t\'usago cxcluet du miniltcre@):H:H::"".F.:i*ij,i;ilTi".,",",irectorBusiness CorPorationsAct /LBi :"r"J\',\'ff ""ti6t6 s p a\' acrionsOnlrrlo Corporutlon Numb•rNum6ro do lr comp6gnio on Ontarlo307421TRANScoo•18Ministero dos Solvicesaux consommateuts?#rd;Lr\'\'+h$"Tno"u\'u"are ellective onJANUAFYz a JAIWIEH, 20tzARTICLES OF AMENDMENTSTATUTS DE MODIFICATIONForm 3BusinessCorporalion3ActFormulenum6ro 3Loisur le3compagni6s1. The present name of the corporation is:GRE Cr 0-rIBE2. The name of the corporation is changed to (ifapplicable):D6nomination sociale actuel

In [19]:
input_phrase = "GRECO" #@param {type:"string"}
#input phrase from user - make interactive using forms 
print(input_phrase)



GRECO


In [20]:
input_phrase_embeddings = model_para_distilroberta.encode(input_phrase)

In [11]:
sentences = file_list

We measure how long it takes for the pretrained model to generate embeddings on our test input 

In [12]:
start_time = time.time()
sentence_embeddings = model_para_distilroberta.encode(sentences)
end_time = time.time()
print("Time taken to compute embeddings for file","extracted.txt using pretrained model", model_name, " is:",  str(end_time - start_time))

Time taken to compute embeddings for file extracted.txt using pretrained model sentence-transformers/paraphrase-distilroberta-base-v2  is: 37.01459836959839


In [13]:
#embeddings for the entire pdf
print(sentences, " : " ,  sentence_embeddings)

["O2COPL)\\pRoTSSORYOTEDue: On Demand Dated: JulY Ist, L975$63,364.25 R. R. #3Woodstock, OntarioON DEMAND, I promise to pay CHRISTOPHER ECONOMOPOULOS----the sum of SIXtY-THREE THOUSAND, THREE HUNDRED AND SIXTY-FOUR---- ($o:,364.25)--- 25/100 DoLLARStogether with interest at the rate of nil per cent perannum. Value receive'd.GRECO-IBERIAN CERAMICS LIMITED(r*,U3Per:ct,ll\n", '!=om(nG)o@\n', 'oo-to\n', '\n', 'For Minartry Ur. OntyA t\'usago cxcluet du miniltcre@):H:H::"".F.:i*ij,i;ilTi".,",",irectorBusiness CorPorationsAct /LBi :"r"J\',\'ff ""ti6t6 s p a\' acrionsOnlrrlo Corporutlon Numb•rNum6ro do lr comp6gnio on Ontarlo307421TRANScoo•18Ministero dos Solvicesaux consommateuts?#rd;Lr\'\'+h$"Tno"u\'u"are ellective onJANUAFYz a JAIWIEH, 20tzARTICLES OF AMENDMENTSTATUTS DE MODIFICATIONForm 3BusinessCorporalion3ActFormulenum6ro 3Loisur le3compagni6s1. The present name of the corporation is:GRE Cr 0-rIBE2. The name of the corporation is changed to (ifapplicable):D6nomination sociale actuelle d

TODO - Distribution of cosine scores across each sentence - use Plotly Dash or Plotly Python Open Soure Graphing Library 

In [14]:
#set a threshold 
#cosine similarity score ranges between 0 and 1 
#0 meaning no similarity and 1 meaning exact match 
#set a threshold .5 to start with 
cosine_similarity(
    [input_phrase_embeddings],
    sentence_embeddings
)

array([[ 0.0497728 ,  0.0333069 , -0.04198862,  0.12559575,  0.15522355,
         0.04741888,  0.33817965,  0.29981408,  0.1504201 ,  0.05450509,
         0.06681477,  0.05318683,  0.05017577,  0.0464457 ,  0.08392965,
         0.07534496,  0.17116067,  0.37096396, -0.05867774, -0.03142027,
        -0.04274528, -0.02227909, -0.00095353,  0.05705285, -0.0228506 ,
         0.04731575,  0.05084439,  0.22083625,  0.26320824,  0.05193338,
         0.13119684,  0.23205215,  0.04888822,  0.10790619,  0.12559575,
         0.12559575,  0.4867283 ,  0.0430879 ,  0.24879912,  0.27119136,
         0.26370424,  0.19146574,  0.35294193,  0.05639128,  0.143294  ,
         0.02565302,  0.15139967,  0.2327385 ,  0.17560324,  0.19739929,
         0.28592932,  0.24226412,  0.09439895,  0.2730418 , -0.01540712,
         0.04759608,  0.31599313,  0.41766828,  0.2703469 ,  0.07035723,
         0.25587654,  0.25214082,  0.06072365,  0.22489297,  0.23132542,
         0.07036918,  0.26627308,  0.05265969,  0.0

Example of input phrase and output of cosine similarity between input phrase and the sentence embedding  of sentence 0 

In [23]:
i = 10
sentence_embeddings = model_para_distilroberta.encode(sentences[i])
cosine_score = cosine_similarity([input_phrase_embeddings], sentence_embeddings.reshape(1,-1))
print("Input Phrase" , input_phrase , "matched at page number:" , i , ": Sentence :", sentences[0] , ":" , cosine_score)
 




Input Phrase GRECO matched at page number: 10 : Sentence : O2COPL)\pRoTSSORYOTEDue: On Demand Dated: JulY Ist, L975$63,364.25 R. R. #3Woodstock, OntarioON DEMAND, I promise to pay CHRISTOPHER ECONOMOPOULOS----the sum of SIXtY-THREE THOUSAND, THREE HUNDRED AND SIXTY-FOUR---- ($o:,364.25)--- 25/100 DoLLARStogether with interest at the rate of nil per cent perannum. Value receive'd.GRECO-IBERIAN CERAMICS LIMITED(r*,U3Per:ct,ll
 : [[0.12725039]]


We set a cosine score threshold of .5

We search through the sentence embeddiings for one that is similar to the input phrase "TROTTER" 

The selection of the optimum cosine similarity is based on trials over various texts.

.5 is a standard choice. 

Record time taken 



In [30]:

start_time = time.time()

for i in range(len(sentences)):
  #print("{} \t\t {} \t\t Score: {:..4f}".format(input_phrase, sentences[i], cosine_similarity([input_phrase_embeddings].reshape(1,-1), sentence_embeddings[i].reshape(1,-1))))
  sentence_embeddings = model_para_distilroberta.encode(sentences[i])
  cosine_score = cosine_similarity([input_phrase_embeddings], sentence_embeddings.reshape(1,-1))
  if (cosine_score > .4):
     print("Input Phrase" , input_phrase , "matched at page number:" , i , ": Sentence :", sentences[i] , ":" , cosine_score)

end_time = time.time()

 

Input Phrase GRECO matched at page number: 42 : Sentence : FIRST RESOLUTTONS OF THE DIRECTOR OF GRECO-IBERIA]{ CERAMTCS I,IMITEDRESOLVED THAT:1. It was noted that Articles of Incorporation had been issuedto the Corporation, dated the 3rd day of June , 1975.2. CHRISTOPHER ECONOMOPOULOS is hereby elected and appointedas President and Secretary-treasurer of the Corporation.3. One common share of the capital stock of the Corp-oration be allotted and issued to Christopher Economopoulospursuant to his strbscription for such share at $I.00 per sharecontained in the articles for incorporation, and that the Corp-oration having received the sum of $f.00 for such share thatthe said common share is hereby declared to be a fulIy-paid andnon-assessable share.4. The form of Common Share Certificate annexed hereto andinitialled by the President for identification be and the sameis hereby approved and adopted as the form of common Share Certi-ficate of the Corporation, and that the form of Preference S

The challenge lies in finding the optimum threshold value over which the extractions could be true matches.

I experimented with .1 , .2 and .3 threshold values.

.1 threshold outputted sentences where there was no mention ofthe input phrase.

The same with .2 and .3 .

.4 threshold outputted only exact matches to the input phrase.




In [31]:
print("Time taken to find the best match for the input phrase in ","extracted.txt using pretrained model", model_name, " is:",  str(end_time - start_time))

Time taken to find the best match for the input phrase in  extracted.txt using pretrained model sentence-transformers/paraphrase-distilroberta-base-v2  is: 41.26996874809265


Can we conclude that SentenceTransformer pretrained model may not be suited to our use case?

Sentence Trans can be used for paraphrase mining, and comparing pieces of text.  




https://www.kaggle.com/Cornell-University/arxiv

https://www.youtube.com/watch?v=4I3gS1cmqe4



Comparing documents using SentenceTransformers

Each document is compared to  itself and remaining documents 


Zero-shot classification using HuggingFace 
https://www.kaggle.com/foolofatook/zero-shot-classification-with-huggingface-pipeline
